In [ ]:
!pip install streamlit

In [ ]:
! wget -w -O - ipv4.icanhazip.com

wget: --wait: Invalid time period ‘-O’


In [ ]:
!pip install langdetect

In [ ]:
!pip install transformers

In [ ]:
%%writefile app.py

import streamlit as st
import transformers
from transformers import MarianMTModel, MarianTokenizer
import langdetect
from langdetect import detect

st.set_page_config(page_title='Language2Language', page_icon=":tada:", layout="wide")
# HEADER
with st.container():
    st.title(":violet[_Language2Language_] :sparkles:")
    st.subheader(""":grey[Welcome to _your automatic language translator_ for FREE]""", divider="orange")


# Upload a file
def load_file():
    file = st.file_uploader(":grey[Select your .txt file and... let's go!]", type= ["txt"], accept_multiple_files=False)
    if file is not None:
        file_content = file.getvalue().decode("utf-8")

        #intermediate_line = int(len(file)/2) #intermediate line of text
        st.write(f"Number of words: {len(file_content)}")
        st.write(file_content[60:500])
        try:
            #file.seek(0)
            #file = file.read()
            content_interm_line =file_content[60:500]
            source_language = detect(content_interm_line)
            st.write("The source language of this text is:", source_language)
            check_src_language= st.text_input("Do you agree with the source language? (y/n)")
            if check_src_language != "y" and check_src_language != "":
              source_language = st.text_input("Which is the original language of your document? (required): English (en), French (fr), Spanish (es)")
            target_language = st.text_input("Into which language you want to translate this text? English (en), French (fr), Spanish (es)")
            if not target_language:
                st.warning("Please select a target language")


            else:
              #target_language = st.text_input("Into which language you want to translate this document? English (en), French (fr), Spanish (es) etc...")

              full_translation = []
              for line in file_content.splitlines():
                translated_sentence = translate_sentence(sentence=line,
                                                   source_language=source_language,
                                                   target_language=target_language
                                                   )
                if translated_sentence is not None:
                  full_translation.append(translated_sentence)

              if full_translation:
                full_translation_str = " ".join(full_translation)

              download_txt_translated_file(full_translation_str, target_language)
              st.stop()


        except Exception as e:
          st.error(f"An error ocurred during source language detection: {e}")


def translate_sentence(sentence, source_language, target_language):
    model = MarianMTModel.from_pretrained(get_model_name(source_language, target_language))
    tokenizer = MarianTokenizer.from_pretrained(get_model_name(source_language, target_language))

    input_ids = tokenizer.encode(sentence, return_tensors="pt")
    translated_tokens = model.generate(input_ids, max_length=512)
    translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_sentence


def get_model_name(source_lang, target_lang):
    model_name = f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}"

    if source_lang == None and target_lang==None:
      st.warning(f"Source language {source_lang} and Target language {target_lang} is not yet implemented. Sorry!")
      st.stop()

    return model_name


def download_txt_translated_file(text_content, target_language):
    if text_content:
        st.download_button("Download your text content!", text_content, file_name=f"Language2Language_{target_language}.txt")
        st.write("\n\n:violet[Thank you! :confetti_ball:]")


load_file()

Overwriting app.py


In [ ]:
!huggingface-cli login

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501